In [14]:
import airbyte as ab
import os
from dotenv import load_dotenv
import json
load_dotenv()
import pandas as pd

In [2]:
# --- CẤU HÌNH "BAO VÂY" ---
config = {
    "host": os.getenv("ORACLE_HOST"),
    "port": int(os.getenv("ORACLE_PORT")),
    "username": os.getenv("ORACLE_USER"),
    "password": os.getenv("ORACLE_PASSWORD"),
    
    # CHIẾN THUẬT 1: Cung cấp cả SID lẫn Service Name để tránh lỗi NullPointer
    # (Dù DB bạn dùng Service Name, ta cứ đẩy giá trị đó vào key 'sid' để lừa Connector)
    "sid": os.getenv("ORACLE_SERVICE_NAME"),          
    "service_name": os.getenv("ORACLE_SERVICE_NAME"), 
    
    "schemas": ["DCC_TAISUN"], 
    
    # CHIẾN THUẬT 2: Thêm cấu hình SSH Tunnel (BẮT BUỘC với nhiều version mới)
    # Nếu thiếu cái này, code Java thường bị crash khi check tunnel.
    "tunnel_method": {
        "tunnel_method": "NO_TUNNEL"
    },

    # CHIẾN THUẬT 3: Unencrypted chuẩn
    "encryption": {
        "encryption_method": "unencrypted"
    }
}

print(">>> Đang gửi cấu hình sau tới Airbyte Connector:")
print(json.dumps(config, indent=2)) 

# Khởi tạo source
full_refresh_source = ab.get_source("source-oracle", config=config)


>>> Đang gửi cấu hình sau tới Airbyte Connector:
{
  "host": "10.0.0.250",
  "port": 1521,
  "username": "DCC_TAISUN",
  "password": "dcc_taisun",
  "sid": "TOPPROD",
  "service_name": "TOPPROD",
  "schemas": [
    "DCC_TAISUN"
  ],
  "tunnel_method": {
    "tunnel_method": "NO_TUNNEL"
  },
  "encryption": {
    "encryption_method": "unencrypted"
  }
}


Writing PyAirbyte logs to file: /tmp/airbyte/logs/2025-12-04/airbyte-log-KBM6K9T69.log
Writing `source-oracle` logs to file: /tmp/airbyte/logs/source-oracle/source-oracle-log-KBM6K9T7B.log


In [3]:
try:
    full_refresh_source.check()
except Exception as e:
    print(e)

Connection check succeeded for `source-oracle`.

In [4]:
full_refresh_available_streams = full_refresh_source.get_available_streams()

In [5]:
des_config={
            "s3_bucket_name": os.getenv("TARGET_BUCKET_NAME", "bronze-v2"),
            "s3_bucket_path": "bronze-v2", # Sub-folder trong bucket
            "s3_bucket_region": "us-east-1", # MinIO không quan trọng region, để default
            "access_key_id": os.getenv("MINIO_ACCESS_KEY"),
            "secret_access_key": os.getenv("MINIO_SECRET_KEY"),
            "s3_endpoint": os.getenv("AIRBYTE_MINIO_ENDPOINT"), # QUAN TRỌNG: Endpoint cho Docker container
            "s3_path_style_access": True,
            "format": {
                "format_type": "Parquet", # Khuyên dùng Parquet cho Data Lake
                "compression_codec": "SNAPPY"
            },
        }

In [6]:
destination = ab.get_destination(
        "destination-s3",
        config=des_config
    )

Writing `destination-s3` logs to file: /tmp/airbyte/logs/destination-s3/destination-s3-log-KBM73EVH3.log


In [7]:
try:
    print("🔌 Đang kiểm tra kết nối MinIO...")
    destination.check()
    print("✅ Kết nối MinIO thành công!")
except Exception as e:
    print(e)

🔌 Đang kiểm tra kết nối MinIO...


Connection check succeeded for `destination-s3`.

✅ Kết nối MinIO thành công!


In [11]:
selected_streams = ["2504_TC_ABB_FILE", "2022_DOMESTIC_SALES"]
full_refresh_source.select_streams(selected_streams)
cursor_config = {
    "2504_TC_ABB_FILE": "TC_ABB01",
    "2022_DOMESTIC_SALES":"AMOUNT"
}
full_refresh_source.set_cursor_keys(**cursor_config)

In [12]:
a = full_refresh_source.get_configured_catalog(streams=selected_streams)
print(a)

streams=[ConfiguredAirbyteStream(stream=AirbyteStream(name='2022_DOMESTIC_SALES', json_schema={'type': 'object', 'properties': {'TARGET': {'type': 'number'}, 'SYSTEM': {'type': 'string'}, 'MONTH_STR': {'type': 'string'}, 'PRODUCT_GRP_ORDER': {'type': 'number'}, 'CUSTOMER_ID': {'type': 'string'}, 'PRODUCT_GRP': {'type': 'string'}, 'BRAND_ORDER': {'type': 'number'}, 'AREA_ORDER': {'type': 'number'}, 'BRAND': {'type': 'string'}, 'AREA': {'type': 'string'}, 'CUSTOMER_NAME': {'type': 'string'}, 'AMOUNT': {'type': 'number'}, 'SYSTEM_ORDER': {'type': 'string'}, 'MONTH_NUM': {'type': 'number'}, 'CAT': {'type': 'string'}, 'CAT_ORDER': {'type': 'number'}, 'TOTAL_AMOUNT': {'type': 'number'}}}, supported_sync_modes=[<SyncMode.full_refresh: 'full_refresh'>, <SyncMode.incremental: 'incremental'>], source_defined_cursor=None, default_cursor_field=[], source_defined_primary_key=[], namespace='DCC_TAISUN', is_resumable=None), sync_mode=<SyncMode.incremental: 'incremental'>, cursor_field=['AMOUNT'], des

In [16]:
dfs = []
for stream_item in selected_streams:
    print(f"-------syns data from table {stream_item}---------")
    records_iterator = full_refresh_source.get_records(stream=stream_item)
    data = pd.DataFrame(list(records_iterator))
    print(f"-------save data---------")
    data.to_csv(f"./data/{stream_item}.csv", index=False, encoding='utf-8-sig')
    dfs.append(data)

    

-------syns data from table 2504_TC_ABB_FILE---------
-------save data---------
-------syns data from table 2022_DOMESTIC_SALES---------
-------save data---------


In [17]:
from deltalake import write_deltalake
import os

In [ ]:
storage_options = {
    "AWS_ACCESS_KEY_ID": os.getenv("MINIO_ACCESS_KEY"),
    "AWS_SECRET_ACCESS_KEY": os.getenv("MINIO_SECRET_KEY"),
    "AWS_ENDPOINT_URL": os.getenv("AIRBYTE_MINIO_ENDPOINT"), # Ví dụ: http://192.168.1.15:9000
    "AWS_REGION": "us-east-1", # Bắt buộc phải có, dù MinIO không dùng
    "AWS_ALLOW_HTTP": "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true" # <--- BẮT BUỘC VỚI MINIO
}

In [31]:
# --- 3. ĐỊNH NGHĨA ĐƯỜNG DẪN ---
# Delta Table là một THƯ MỤC, không phải 1 file đơn lẻ
bucket_name = "bronze-v2"
table_name = "2022_DOMESTIC_SALES"
s3_path = f"s3://{bucket_name}/delta_lake/{table_name}"
print(f"🚀 Đang ghi Delta Table xuống: {s3_path}")

🚀 Đang ghi Delta Table xuống: s3://bronze-v2/delta_lake/2022_DOMESTIC_SALES


In [32]:
try:
    write_deltalake(
        s3_path,
        dfs[0],
        mode="overwrite", # Hoặc 'append' nếu muốn nối thêm
        storage_options=storage_options
    )
    print("🎉 THÀNH CÔNG! Đã lưu dưới dạng Delta Format.")
    
except Exception as e:
    print(f"❌ Lỗi ghi Delta: {e}")

❌ Lỗi ghi Delta: Kernel error: Error interacting with object store: Generic S3 error: Error performing GET http://172.16.15.29:9000/bronze-v2/delta_lake/2022_DOMESTIC_SALES/_delta_log/_last_checkpoint in 169µs - HTTP error: builder error


In [ ]:
cachecache = ab.get_default_cache()

result = full_refresh_source.read(cache=cachecache, streams=selected_streams, force_full_refresh=True)

------------------------------------------------

### Sync Progress: `source-oracle -> DuckDBCache`

**Started reading from source at `14:38:02`:**

- Read **1,996,382** records over **7min 12s** (4,617.3 records/s, 6.99 MB/s).

- Received records for 1 streams:
  - 1,996,382 CCH_FILE

- Cached **1,996,382** records into 20 local cache file(s).

- Finished reading from source at `14:45:33`.

**Started cache processing at `14:45:33`:**

- Processed **20** cache file(s) over **18 seconds**.

- Completed cache processing for 1 streams:
  - CCH_FILE

- Finished cache processing at `14:45:51`.






**Sync completed at `14:45:51`. Total time elapsed: 7min 49s**

------------------------------------------------


Completed `source-oracle -> DuckDBCache` sync at `07:45:51` (`CCH_FILE` stream).


In [ ]:
df = result.cache.streams[selected_streams[0]].to_pandas()